# PatchCore Transformed

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [3]:
import importlib

import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import DataLoader

import Thesis_dev
import Utility
from Thesis_dev import MVTecDataset, PatchCoreViT, VanillaPatchCore, PatchCoreSWin

In [ ]:
folder_name = "/content/drive/MyDrive/bottle_xs"

train_path, test_path = os.path.join(folder_name, "train", "good"), os.path.join(folder_name, "test")
train_paths, test_paths = [train_path], [ os.path.join(test_path, anomaly) for anomaly in os.listdir(test_path) if anomaly[0] != '.']

# Examples

## Vanilla

In [ ]:
importlib.reload(Thesis_dev)
importlib.reload(Utility)

backbone = "timm/wide_resnet50_2.tv_in1k"
layers = ["layer2", "layer3"]

pc = VanillaPatchCore(layers=layers, backbone=backbone, f_coreset=1)
pc.fit(train_paths)

pc.evaluate(test_paths, validation_flag = True)

[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


Val: IMAGE Level ROCAUC: 1.0
Val: PIXEL Level ROCAUC: 0.9523194638170911
[INFO][evaluate] Image Level ROCAUC: 1.000
[INFO][evaluate] Initial Score Threshold: 2.922 F1Score: 0.889
[INFO][evaluate] Optimal Score Threshold: 1.522 F1Score: 1.000
[INFO][evaluate] Average Inference time with batch_size=1: 0.082s


In [ ]:
filepath = "/content/drive/MyDrive/bottle_xs/test/broken_large/000.png"
img = Image.open(filepath).convert("RGB")
sample = pc.processor(img)
sample_torch = torch.Tensor(sample['pixel_values'][0]).unsqueeze(0)
sample['pixel_values'][0]=sample_torch

temp = pc.extract_embeddings(sample_torch)

## ViT

In [ ]:
# Config
backbone = "google/vit-base-patch16-224-in21k"
layers = [4]

# Training ViT
pcViT = PatchCoreViT(layers=layers, backbone=backbone, f_coreset=1)
pcViT.fit(train_paths)

# Testing
pcViT.evaluate(test_paths, validation_flag = True)

In [ ]:
# Predict one sample
filepath = "/content/drive/MyDrive/bottle_xs/test/broken_large/000.png"
img = Image.open(filepath).convert("RGB")
sample = pcViT.processor(img)
sample_torch = torch.Tensor(sample['pixel_values'][0]).unsqueeze(0)
sample['pixel_values'][0]=sample_torch
score, segm_map = pcViT.predict(sample)

# plot
plt.imshow((cv2.resize(np.array(img), (224,224))))
plt.imshow(segm_map.reshape(224,224), alpha=0.7)
plt.show()

In [ ]:
importlib.reload(Utility)
Utility.create_gif(filepath, pcViT)

## SWin

In [ ]:
importlib.reload(Thesis_dev)
importlib.reload(Utility)

from Thesis_dev import PatchCoreSWin

backbone = "microsoft/swin-small-patch4-window7-224"

pc = PatchCoreSWin(layers=[2], blocks=[0,1,2,3], backbone=backbone, f_coreset=1)
pc.fit(train_paths)

pc.evaluate(test_paths, validation_flag = True)

# Evaluations

In [4]:
! pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=6872bff956088e6c9c11c70b2b5c85812abc721c18be5008a798978f80997b0b
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [5]:
import wget
import shutil

In [6]:
class_links = {
    "bottle": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz",
    "cable": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937413-1629951498/cable.tar.xz",
    "capsule": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz",
    "carpet": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937484-1629951672/carpet.tar.xz",
    "grid": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937487-1629951814/grid.tar.xz",
    "hazelnut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937545-1629951845/hazelnut.tar.xz",
    "leather": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937607-1629951964/leather.tar.xz",
    "metal_nut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937637-1629952063/metal_nut.tar.xz",
    "pill": "https://www.mydrive.ch/shares/43421/11a215a5749fcfb75e331ddd5f8e43ee/download/420938129-1629953099/pill.tar.xz",
    "screw": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938130-1629953152/screw.tar.xz",
    "tile": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938133-1629953189/tile.tar.xz",
    "toothbrush": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938134-1629953256/toothbrush.tar.xz",
    "transistor": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938166-1629953277/transistor.tar.xz",
    "wood": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938383-1629953354/wood.tar.xz",
    "zipper": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938385-1629953449/zipper.tar.xz"
}

In [7]:
# Download duration 6m 5s
for key in list(class_links.keys())[0:2]: # TODO_CHANGE
  os.mkdir(key)
  file_path = wget.download(class_links[key], key)
  if os.path.exists(file_path):
    # Extract the file if it's a tar.xz file
    if file_path.endswith('.tar.xz'):
      shutil.unpack_archive(file_path, extract_dir=key)
      os.remove(file_path)
      print(f"File {key} downloaded and extracted successfully.")
    else:
      print(f"Failed to download the file {key}.")

File bottle downloaded and extracted successfully.
File cable downloaded and extracted successfully.


In [ ]:
# remove
# shutil.rmtree("bottle")
# importlib.reload(patchcore_dev)

In [34]:
def print_results(results):
  print("\nCLASS BREAKDOWN")
  misclassified = average = 0
  for className in results.keys():
    result = results[className]
    misclassified = misclassified + result["cm"][0][1] + result["cm"][1][0]
    average = average + result['auc']
    print(f"ROCAUC: {result['auc']:.3f} \t\tf1_score: {result['prfs'][2]:.3f} \t{className}")

  print()
  average = average/len(results.keys())

  print("\nSUMMARY")
  print(f"Avg AUC:{average:.3f} \t\t Total Misclassified={misclassified}")

## Vanilla


Single Layer Perfomance

In [35]:
importlib.reload(Thesis_dev)
importlib.reload(Utility)

from Thesis_dev import VanillaPatchCore

layers = ["layer1", "layer2", "layer3", "layer4"]
backbone = "timm/wide_resnet50_2.tv_in1k"
f_coreset = 0.0062

for layer in layers:
  print()
  print(f"{layer}")
  results = {}
  base_path = "/content/"

  for key in list(class_links)[0:2]: # TODO_CHANGE
    print()
    print(key)
    temp_path = os.path.join(base_path, key, key)

    train_path, test_path  = os.path.join(temp_path, "train", "good"), os.path.join(temp_path, "test")
    train_paths, test_paths= [train_path], [os.path.join(test_path,path) for path in os.listdir(test_path)]

    # Train & Evaluate
    model = VanillaPatchCore(layers=[layer], backbone=backbone, f_coreset=f_coreset)
    model.fit(train_paths)
    model.evaluate(test_paths, validation_flag = True)

    # Save model results
    temp_result = {}
    temp_result["cm"] = model.cm
    temp_result["prfs"] = model.prfs
    temp_result["auc"] = model.auc

    results[key] = temp_result

  # Print results
  print_results(results)


layer1

bottle
[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 18.06it/s]


Error: could not project vectors. Please increase `eps`.
Start Coreset Subsampling...


100%|██████████| 83/83 [00:04<00:00, 18.82it/s]


Val: IMAGE Level ROCAUC: 1.0
Val: PIXEL Level ROCAUC: 0.9545156092161134
[INFO][evaluate] Image Level ROCAUC: 1.000
[INFO][evaluate] Initial Score Threshold: 0.957 F1Score: 0.992
[INFO][evaluate] Optimal Score Threshold: 0.947 F1Score: 1.000
[INFO][evaluate] Average Inference time with batch_size=1: 0.017s

cable
[INFO][__init__] Model PatchCore loaded on device: cuda


...

Val: IMAGE Level ROCAUC: 1.0
Val: PIXEL Level ROCAUC: 0.9665774067837939
[INFO][evaluate] Image Level ROCAUC: 1.000
[INFO][evaluate] Initial Score Threshold: 1.369 F1Score: 0.992
[INFO][evaluate] Optimal Score Threshold: 1.179 F1Score: 1.000
[INFO][evaluate] Average Inference time with batch_size=1: 0.021s

cable
[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 224/224 [00:16<00:00, 13.53it/s]


Start Coreset Subsampling...


 79%|███████▉  | 862/1087 [00:04<00:01, 189.06it/s]


KeyboardInterrupt: 